In [0]:
%run /Workspace/Users/prabhas.kota@c2stechs.com/Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
file=dbutils.fs.ls(f"{dataset_bookstore}/orders-raw")
display(file)

In [0]:
(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation","dbfs:/FileStore/tables/schema")
    .load(f"{dataset_bookstore}/orders-raw")
    .createOrReplaceTempView("orders_raw_temp")
)


In [0]:
%sql
SELECT count(*) FROM orders_raw_temp

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW orders_temp as(
    select *, current_timestamp()  arrival_time ,input_file_name() source_file   
    from orders_raw_temp
)

In [0]:
%sql
SELECT * FROM orders_temp

In [0]:
    (spark.table("orders_temp")
        .writeStream
        .format("delta")
        .option("checkpointLocation", "dbfs:/FileStore/tables/orders_checkpoint/bronze")
        .outputMode("append")
        .table("orders_bronze")
    )
    

In [0]:
%sql
select count(*) from orders_bronze

In [0]:
load_new_data()

In [0]:
(spark.read
    .format("json")
    .load(f"{dataset_bookstore}/customers-json")
    .createOrReplaceTempView("customers_lookup")
)

In [0]:

(spark.readStream
    .table('orders_bronze')
    .createOrReplaceTempView('Orders_enriched_temp')
)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW Orders_Customers_temp
AS
  SELECT 
  order_id,
  quantity,
  o.customer_id, 
  c.profile:first_name  f_name,
  c.profile:last_name as l_name,
  cast(from_unixtime(order_timestamp,'yyyy-MM-dd HH:mm:ss') as timestamp) as order_timestamp,
  books
  FROM Orders_enriched_temp o
  INNER JOIN customers_lookup c
  ON C.customer_id = O.customer_id
  WHERE QUANTITY > 0


In [0]:
(spark
    .table("Orders_Customers_temp")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "dbfs:/FileStore/tables/orders_checkpoint/silver")
    .outputMode("append")
    .table("orders_silver")
)

In [0]:
(spark.readStream
    .table('orders_silver')
    .createOrReplaceTempView('orders_silver_temp')
)



In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW orders_enriched_silver
AS(
  SELECT customer_id, f_name AS FirstName,l_name as LastName,date_trunc('DD',order_timestamp) as order_date,sum(quantity) as book_count
  FROM 
  orders_silver_temp
  GROUP BY customer_id, FirstName,LastName,order_date
)


In [0]:
(spark.table("orders_enriched_silver")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "dbfs:/FileStore/tables/orders_checkpoint/gold")
    .outputMode("complete")
    .trigger(availableNow=True)
    .table("orders_gold")
)

In [0]:
%sql
select * from orders_gold

In [0]:
for s in spark.streams.active:
  print("stoped"+s.id)
  s.stop()
  s.awaitTermination()